<a href="https://colab.research.google.com/github/cloudhood/learning-python/blob/main/notebooks/iterators.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Iterator protocol
The for loop first checks if the object `s` is referring to is **iterable** - can it iterate over it. If it is, each time it asks for the next thing, and it's assigned `_` each time. When the loop reaches the end of the loop, it exists.
1. Are you iterable?
2. Get the next value from the object from each iteration.
3. Exit the loop at the end.

In [201]:
# Notice: there is no index or incrementing.
s = "abcde"

print("Start")
for _ in s:
  print(_)
print("End")

Start
a
b
c
d
e
End


In [9]:
i = iter(s)
i

In [20]:
def _for(x):
  i = iter(x)
  while True:
    try:
      print(next(i))
    except StopIteration:
      break

In [21]:
_for(s)

a
b
c
d
e


In [26]:
l = [1, 2, 3, 4]
d = {"a": 1, "b": 2, "c": 3}

In [27]:
_for(l)

1
2
3
4


In [28]:
_for(d)

a
b
c


In [29]:
_for(d.items())

('a', 1)
('b', 2)
('c', 3)


In [30]:
# https://stackoverflow.com/questions/54387889/how-does-the-python-for-loop-actually-work
import dis
dis.dis("for _ in s: pass")

  1           0 SETUP_LOOP              12 (to 14)
              2 LOAD_NAME                0 (s)
              4 GET_ITER
        >>    6 FOR_ITER                 4 (to 12)
              8 STORE_NAME               1 (_)
             10 JUMP_ABSOLUTE            6
        >>   12 POP_BLOCK
        >>   14 LOAD_CONST               0 (None)
             16 RETURN_VALUE


## Iterable vs Iterator
* Iterable: When we run `iter()` on the object it returns an `iterator` obeject rather than raising a `TypeError`. It can be put into a `for` loop: strings, lists, tuples, dicts, sets, files, etc. 
* Iterator: Object on which `next()` is run. Not necessarily the original object, but it can be. This is what is being invoked on each iteration, and raises a `StopIteration` at the end. 

In [32]:
iter(s)

In [33]:
iter(l)

In [34]:
iter(d)

In [35]:
iter(d.items())

In [38]:
## Printed representation shows address in memory - notice how new iterators are
## returned with each invocation of `iter()`.
iter(s)

In [37]:
iter(s)

# Iterator Protocol

1. Must respond to `iter()`, returning an `iterator` object.
2. The returned  `iterator` must return something new with each invocation of `next()`.
3. When there are no more objects to return, then the `iterator` should raise `StopIteration`.

How can we add this protocol to our own objects?

In [40]:
class MyIterator():
  def __init__(self, data):
      self.data = data

# New instance of MyIterator
m = MyIterator("abc")

# The iterator object determines what is returned in each iteration
for _ in m:
  print(_)

TypeError: ignored

We need to make `MyIterator` **iterable**. `iterable` implements `__iter__` while `iterator` implements `__next__`

In [44]:
class MyIterator():
  def __init__(self, data):
      print("In __init__")
      self.data = data
      self.index = 0 # Keep track of where we are in our object
  # For loop will call iter() once.
  def __iter__(self):
      print("In __iter__")
      return self    
  
  # Invoked each time next() is run.
  def __next__(self):
      print(f"In __next__,\nCurrent index: {self.index}")
      if self.index >= len(self.data):
          print("Raising StopIteration")
          raise StopIteration
      value = self.data[self.index]
      self.index += 1
      print(f"Incremented index: {self.index}")
      return value

# New instance of MyIterator
m = MyIterator("abc")

# The iterator object determines what is returned in each iteration
for _ in m:
  print(_)

In __init__
In __iter__
In __next__,
Current index: 0
Incremented index: 1
a
In __next__,
Current index: 1
Incremented index: 2
b
In __next__,
Current index: 2
Incremented index: 3
c
In __next__,
Current index: 3
Raising StopIteration


In [45]:
for _ in range(5):
    print(_)

0
1
2
3
4


## Exercise 1
Write an iterator `Circle` that takes two arguments, an iterable and an integer. The integer indicates how many elements you'll get back from the iterator.  The elements will be taken from the iterable.  If the iterable is too short, then the iterator will return to the start.

So if we say:

    c = Circle('abcd', 7)

We can then say:

    for one_item in c:
        print(one_item)

And we'll get:

    a
    b
    c
    d
    a
    b
    c



In [149]:
class Circle1():
    def __init__(self, data, maxtimes):
        self.data = data
        self.maxtimes = maxtimes
        self.index = 0

    def __iter__(self):
        return self

    def __next__(self):
        if self.index == self.maxtimes:
            raise StopIteration
        
        val = self.data[self.index % len(self.data)]
        self.index += 1

        return val

In [150]:
class Circle2():
    def __init__(self, data, elts):
        self.data = data
        self.elts = elts
        self.index = 0
        self.counter = 0

    ## Object itself is the iterator
    def __iter__(self):
        return self

    def __next__(self):
        if (self.index == len(self.data)) :
            self.index = 0
        if self.counter == self.elts:
            raise StopIteration
        
        val = self.data[self.index]
        self.index += 1
        self.counter += 1

        return val

In [151]:
c1 = Circle1('abcd', 7)
c2 = Circle2('abcd', 7) 

In [153]:
for _ in c1:
    print(_)

a
b
c
d
a
b
c


In [154]:
for _ in c2:
    print(_)

a
b
c
d
a
b
c


## Iterators as filters

In [179]:
class OnlyVowels():
    def __init__(self, data):
        self.data = data
        self.index = 0
    
    def __iter__(self):
        return self

    def __next__(self):
        while True:
            if self.index == len(self.data):
                raise StopIteration
            
            val = self.data[self.index]
            self.index += 1

            if val in "aeiou":
                break
        return val

In [180]:
ov = OnlyVowels("this is a test")

In [181]:
for _ in ov:
    print(_)

i
i
a
e


## Exercise 2
Write an iterator, `EvenlyDivisible`, that takes two arguments:  A list of integers, and an integer.  Only return those list elements that are evenly divisible by the integer.  For example:

    e = EvenlyDivisible([2,3,4,5,6,7,8,9], 2)

    for one_item in e:
        print(one_item)

    2
    4
    6
    8

(2) Write an iterator, `MyRange`, that takes one, two, or three arguments.  It should function like the built-in "range" function:

    for one_item in MyRange(5):
        print(one_item, end=' ')

    0 1 2 3 4

    for one_item in MyRange(5, 10):
        print(one_item, end=' ')

    5 6 7 8 9

    for one_item in MyRange(5, 20, 3):
        print(one_item, end=' ')

    5 8 11 14 17


In [182]:
class EvenlyDivisible():
    def __init__(self, data, n):
        assert isinstance(data, list)
        assert isinstance(n, int)
        self.data = data
        self.n = n
        self.index = 0

    def __iter__(self):
        return self

    def __next__(self):
        while True:
            if self.index == len(self.data):
                raise StopIteration
            val = self.data[self.index]
            self.index += 1
            if val % self.n == 0:
                break
        return val    

In [203]:
e = EvenlyDivisible([2,3,4,5,6,7,8,9], 2)

for one_item in e:
    print(one_item)

2
4
6
8


In [218]:
class MyRange():
    def __init__(self, start, end=None, step=1):
        if end is None:
            self.end = start
            self.start = 0
        else:
            self.start = start
            self.end = end
        self.step = step
    
    def __iter__(self):
        return self
    
    def __next__(self):
        if self.step >= 0:
            if self.start >= self.end:
                raise StopIteration
        else:
            if self.start <= self.end:
                raise StopIteration
        val = self.start
        self.start += self.step
        return val

In [219]:
for _ in MyRange(5):
    print(_, end = " ")

0 1 2 3 4 

In [220]:
for _ in MyRange(5, 10):
    print(_, end = " ")

5 6 7 8 9 

In [221]:
for _ in MyRange(5, 20, 3):
    print(_, end = " ")

5 8 11 14 17 

In [223]:
for _ in MyRange(10, 0, -3):
    print(_, end = " ")

10 7 4 1 